In [48]:
import pandas as pd

bunch = pd.read_csv("./imdb250.csv")

In [49]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import time

class LemmaTokenizer:
    def  __init__(self):
        self.nlp = spacy.load("en")
    def __call__(self, text):
        return [
            token.lemma_ for token in self.nlp(text)
            if not (token.is_stop or token.is_punct or token.is_digit)
        ]

start_time = time.time()

vec = TfidfVectorizer(max_features=50_000, tokenizer=LemmaTokenizer())
data = \
    bunch.Title + " " + bunch.Crew + " " + bunch.Plot + " " + \
    bunch.Tags + " " + bunch.Country + " " + bunch.Reviews

# print([i for i, x in enumerate(data.isna()) if x == True])

features = vec.fit_transform(data)
print(f'Training time {time.time() - start_time}s, features = {len(vec.get_feature_names())}')

Training time 115.29158687591553s, features = 30968


In [50]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [51]:
start_time = time.time()

input_texts = ["movie about banker that killed wife"]
input_features = vec.transform(input_texts)

D, N = knn.kneighbors(input_features, n_neighbors=5, return_distance=True)

for input_text, distances, neighbors in zip(input_texts, D, N):
    print("Input text = ", input_text, "\n")
    for dist, neighbor_idx in zip(distances, neighbors):
        print(bunch.Title[neighbor_idx])
        print("Distance = ", dist, "Neighbor idx = ", neighbor_idx)
        print(bunch.Plot[neighbor_idx][:200])
        print(bunch.Crew[neighbor_idx])
        print("-"*200)
    print("="*200)
    print()
print(f'Query execution time - {time.time() - start_time:.5f} sec')


Input text =  fat people in space 

2001: A Space Odyssey
Distance =  0.8603749375853135 Neighbor idx =  89
"2001" is a story of evolution. Sometime in the distant past, someone or something nudged evolution by placing a monolith on Earth (presumably elsewhere throughout the universe as well). Evolution the
Stanley Kubrick (dir.), Keir Dullea, Gary Lockwood, Stanley Kubrick, Arthur C. Clarke
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Interstellar
Distance =  0.9156629193896471 Neighbor idx =  28
Earth's future has been riddled by disasters, famines, and droughts. There is only one way to ensure mankind's survival: Interstellar travel. A newly discovered wormhole in the far reaches of our sola
Christopher Nolan (dir.), Matthew McConaughey, Anne Hathaway, Jonathan Nolan, Christopher Nolan
--------------------------------------------